In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
import pandas as pd

In [2]:
file = "Indian-Summers-Over-the-years.csv"
spark = SparkSession.builder.appName('Dataframe').getOrCreate()
df_temp_country = spark.read.option("header", True).csv(file)

In [3]:
df_temp_country.show()

+----------+-------------------+-----------------------------+-------+
|        dt| AverageTemperature|AverageTemperatureUncertainty|Country|
+----------+-------------------+-----------------------------+-------+
|1743-11-01| 4.3839999999999995|                        2.294|  Åland|
|1743-12-01|               null|                         null|  Åland|
|1744-01-01|               null|                         null|  Åland|
|1744-02-01|               null|                         null|  Åland|
|1744-03-01|               null|                         null|  Åland|
|1744-04-01|               1.53|                         4.68|  Åland|
|1744-05-01|  6.702000000000001|                        1.789|  Åland|
|1744-06-01| 11.609000000000002|                        1.577|  Åland|
|1744-07-01|             15.342|                         1.41|  Åland|
|1744-08-01|               null|                         null|  Åland|
|1744-09-01|             11.702|                        1.517|  Åland|
|1744-

In [4]:
india_data = df_temp_country.where(func.lower(func.col("Country")) == "india").where(df_temp_country.dt >= "1950-01-01")
india_data.show()

+----------+------------------+-----------------------------+-------+
|        dt|AverageTemperature|AverageTemperatureUncertainty|Country|
+----------+------------------+-----------------------------+-------+
|1950-01-01|17.317999999999998|                         0.12|  India|
|1950-02-01|            18.305|          0.14800000000000002|  India|
|1950-03-01|22.933000000000003|                        0.214|  India|
|1950-04-01|             27.13|                        0.246|  India|
|1950-05-01|            30.267|                        0.248|  India|
|1950-06-01|             29.23|                        0.165|  India|
|1950-07-01|26.581999999999997|                         0.16|  India|
|1950-08-01|26.194000000000006|                        0.233|  India|
|1950-09-01|            25.711|          0.08800000000000001|  India|
|1950-10-01|            23.842|                        0.155|  India|
|1950-11-01|            19.818|                        0.236|  India|
|1950-12-01|        

In [17]:
india_data = india_data.withColumn("AverageTemperature", func.round(func.col("AverageTemperature"),2)).withColumn("AverageTemperatureUncertainty", func.round(func.col("AverageTemperatureUncertainty"),2)).withColumn("year", func.year(func.col("dt")))

india_data_grouped = india_data.groupby(func.col("year")).agg(func.round(func.avg(func.col("AverageTemperature")), 2).alias("AverageTemperature")).sort(func.col("year"))
india_data_grouped.show()

+----+------------------+
|year|AverageTemperature|
+----+------------------+
|1950|             23.66|
|1951|             24.22|
|1952|             24.37|
|1953|             24.56|
|1954|             24.17|
|1955|             23.95|
|1956|              23.9|
|1957|             24.03|
|1958|             24.68|
|1959|             24.27|
|1960|             24.29|
|1961|             23.82|
|1962|             23.89|
|1963|              24.1|
|1964|             24.07|
|1965|             24.09|
|1966|             24.44|
|1967|             23.99|
|1968|             23.87|
|1969|              24.5|
+----+------------------+
only showing top 20 rows



In [18]:
india_data_pd = india_data_grouped.toPandas()